### Instalación de dependencias
Instala las librerías `pandas` y `openpyxl` para poder leer los datos de la tabla en formato excel que se aporta con la presente notebook. Puedes hacerlo ejecutando la siguiente celda directamente (asegúrate de haber inicializado tu entorno virtual de python para esta asignatura).

In [ ]:
%pip install pandas
%pip install openpyxl

In [1]:
import pandas as pd
import numpy as np

### Carga los datos recogidos en el fichero `datos_porfolio_optimization.xlsx`

In [2]:
datos = pd.read_excel('datos_porfolio_optimization.xlsx',index_col=0)

In [3]:
datos

,Coste,ROI
Empresa,,
Empresa A,5,8
Empresa B,1,2
Empresa C,2,1
Empresa D,6,5
Empresa E,7,8
Empresa F,3,2
Empresa G,6,2
Empresa H,8,7
Empresa I,2,6


Definimos
$x_i=1$ si y sólo si el objecto $i$ es seleccionado; $v_i$ es el valor que aporta el recurso $i$ y $c_i$, su coste. Además, $B$ es el presupuesto total del que se dispone. Entonces, el problema de la mochila se expresa como:

$$\max \sum_i x_i v_i$$
$$s.t.\; \sum x_i c_i \leq B$$

Para facilitar su resolución, primero vamos a simpliicar la restricción $\sum x_i c_i \leq B \rightarrow \sum x_i c_i = B$. Esto se corresponde con imponer que sólo se podrán elegir conjuntos de activos cuyo valor total sea exactamente igual al presupuesto disponible (lo cual no es una situación totalmente realista).

**Implementa la función QUBO asociada a este problema y resuélvela con un solver de DWave (clásico, híbrido o cuántico).**


Es aconsejable implementar por separado las contribuciones de la función de coste y las de las restricciones. Así podremos evaluar el coste real de las soluciones proporcionadas por el solver, y determinar si es una solución factible. 

In [4]:
# Implementa la función generate_cost_matrix(datos) que genera la matriz correspondiente con la función de coste
def generate_cost_matrix(datos):
    Q_linear = np.zeros((len(datos),len(datos)))
    # Tu código aquí	
    for i,empresa in enumerate(datos.index):
        Q_linear[i,i] = -datos.loc[empresa,'ROI']

    return Q_linear

In [5]:
# Implementa la función generate_constraint_matrix(datos,presupuesto) que genera la matriz de restricciones
def generate_constraint_matrix(datos,presupuesto):
    Q_constraint = np.zeros((len(datos),len(datos)))
    # Tu código aquí
    # Añade la restricción de presupuesto
    for i,empresa in enumerate(datos.index):
        Q_constraint[i,i] -= 2*presupuesto*datos.loc[empresa,'Coste']
        for j,empresa2 in enumerate(datos.index):
            Q_constraint[i,j] += datos.loc[empresa,'Coste']*datos.loc[empresa2,'Coste']

    return Q_constraint

### Crea una función que compruebe sin una solución es factible (=que cumple las restricciones)
Dada una solución (un vector de 0s y 1s), escribe una función que devuelva el valor `True` si es una solución factible (= si se cumple la restricción), y `False` en caso contrario.

In [6]:
def check_constraint(Q_constraint, x, presupuesto):
    factible = False
    # Tu código aquí
    # Comprueba que se cumple la restricción de presupuesto

    return x@Q_constraint@x == -presupuesto*presupuesto

In [7]:
def interpret_solution(x,datos):
    empresas = []
    for i,empresa in enumerate(datos.index):
        if x[i] == 1:
            empresas.append(empresa)
    return empresas

In [8]:
from dimod import ExactSolver

In [9]:
lagrange_multiplier = 10
presupuesto = 20
Q_linear = generate_cost_matrix(datos)
Q_constraint = generate_constraint_matrix(datos,presupuesto)
qubo = Q_linear + lagrange_multiplier*Q_constraint

In [10]:
exact_solver = ExactSolver()
exact_solver.sample_qubo(qubo).first.sample

{0: np.int8(1),
 1: np.int8(1),
 2: np.int8(1),
 3: np.int8(0),
 4: np.int8(1),
 5: np.int8(1),
 6: np.int8(0),
 7: np.int8(0),
 8: np.int8(1)}

In [11]:
sol = np.array([(val) for val in exact_solver.sample_qubo(qubo).first.sample.values()])
interpret_solution(sol,datos)

['Empresa A', 'Empresa B', 'Empresa C', 'Empresa E', 'Empresa F', 'Empresa I']

In [12]:
check_constraint(Q_constraint, np.array([(val) for val in exact_solver.sample_qubo(qubo).first.sample.values()]), presupuesto)

np.True_

### Crea una función que genere la matriz qubo correspondiente con la restricción de desigualdad de presupuesto
$$\sum_i coste_i \cdot x_i \leq presupuesto$$

In [13]:
def generate_inequality_constraint_matrix(datos,presupuesto):
    # Calcula la cota superior del slack
    bound_slack = presupuesto-min(datos['Coste'])

    # Calcula el número de bits necesarios para representar el slack en binario
    slack_binary = np.ceil(np.log2(bound_slack)).astype(int)
    n = len(datos)+slack_binary

    # Es posible que lo más fácil sea añadir el slack como una nueva variable
    # con coste 2^j para j=0,1,...,slack_binary
    Coste = datos['Coste'].to_list() + [2**j for j in range(slack_binary)]

    Q_constraint = np.zeros((n,n))  
    for i in range(n):
        Q_constraint[i,i] -= 2*presupuesto*Coste[i]
        for j in range(n):
            Q_constraint[i,j] += Coste[i]*Coste[j]

    return Q_constraint

Q_inequality = generate_inequality_constraint_matrix(datos,presupuesto=10)

#### Para el término lineal puedes reutilizar tu implementación anterior

In [14]:
Q_linear_inequality = np.zeros(Q_inequality.shape)

In [15]:
Q_linear_inequality[:len(datos),:len(datos)] = generate_cost_matrix(datos)

### ¿Tienes que modificar tu función de comprobación de la restricción?

### Resuelve el problema con la restricción de desigualdad y analiza la factibilidad de las soluciones devueltas por el solver

In [16]:
import dimod
sampler = dimod.ExactSolver()
sampler.sample_qubo(Q_inequality+Q_linear_inequality).first.sample

{0: np.int8(1),
 1: np.int8(1),
 2: np.int8(1),
 3: np.int8(0),
 4: np.int8(0),
 5: np.int8(0),
 6: np.int8(0),
 7: np.int8(0),
 8: np.int8(1),
 9: np.int8(0),
 10: np.int8(0),
 11: np.int8(0),
 12: np.int8(0)}

In [17]:
import dimod

### Resolución con `dimod.BinaryQuadraticModel`

In [18]:
bqm = dimod.BinaryQuadraticModel({},{},0,dimod.BINARY)

# Añade la función de coste
for i in range(len(datos)):
    bqm.add_variable(i,-datos.loc[datos.index[i],'ROI'])

# Añade la restricción de presupuesto
terms = [(i,datos.loc[datos.index[i],'Coste']) for i in range(len(datos))]
bqm.add_linear_equality_constraint(terms, constant=-presupuesto, lagrange_multiplier=lagrange_multiplier)

In [19]:
bqm.to_qubo()
# bqm.to_numpy_matrix()

({(1, 0): np.float64(100.0),
  (2, 0): np.float64(200.0),
  (2, 1): np.float64(40.0),
  (3, 0): np.float64(600.0),
  (3, 1): np.float64(120.0),
  (3, 2): np.float64(240.0),
  (4, 0): np.float64(700.0),
  (4, 1): np.float64(140.0),
  (4, 2): np.float64(280.0),
  (4, 3): np.float64(840.0),
  (5, 0): np.float64(300.0),
  (5, 1): np.float64(60.0),
  (5, 2): np.float64(120.0),
  (5, 3): np.float64(360.0),
  (5, 4): np.float64(420.0),
  (6, 0): np.float64(600.0),
  (6, 1): np.float64(120.0),
  (6, 2): np.float64(240.0),
  (6, 3): np.float64(720.0),
  (6, 4): np.float64(840.0),
  (6, 5): np.float64(360.0),
  (7, 0): np.float64(800.0),
  (7, 1): np.float64(160.0),
  (7, 2): np.float64(320.0),
  (7, 3): np.float64(960.0),
  (7, 4): np.float64(1120.0),
  (7, 5): np.float64(480.0),
  (7, 6): np.float64(960.0),
  (8, 0): np.float64(200.0),
  (8, 1): np.float64(40.0),
  (8, 2): np.float64(80.0),
  (8, 3): np.float64(240.0),
  (8, 4): np.float64(280.0),
  (8, 5): np.float64(120.0),
  (8, 6): np.floa

In [20]:
bqm.add_linear_inequality_constraint(terms,constant=-presupuesto, label='ineq',lb=0,ub=0,lagrange_multiplier=10.0)

[]

In [21]:
bqm.to_qubo()

({(1, 0): np.float64(200.0),
  (2, 0): np.float64(400.0),
  (2, 1): np.float64(80.0),
  (3, 0): np.float64(1200.0),
  (3, 1): np.float64(240.0),
  (3, 2): np.float64(480.0),
  (4, 0): np.float64(1400.0),
  (4, 1): np.float64(280.0),
  (4, 2): np.float64(560.0),
  (4, 3): np.float64(1680.0),
  (5, 0): np.float64(600.0),
  (5, 1): np.float64(120.0),
  (5, 2): np.float64(240.0),
  (5, 3): np.float64(720.0),
  (5, 4): np.float64(840.0),
  (6, 0): np.float64(1200.0),
  (6, 1): np.float64(240.0),
  (6, 2): np.float64(480.0),
  (6, 3): np.float64(1440.0),
  (6, 4): np.float64(1680.0),
  (6, 5): np.float64(720.0),
  (7, 0): np.float64(1600.0),
  (7, 1): np.float64(320.0),
  (7, 2): np.float64(640.0),
  (7, 3): np.float64(1920.0),
  (7, 4): np.float64(2240.0),
  (7, 5): np.float64(960.0),
  (7, 6): np.float64(1920.0),
  (8, 0): np.float64(400.0),
  (8, 1): np.float64(80.0),
  (8, 2): np.float64(160.0),
  (8, 3): np.float64(480.0),
  (8, 4): np.float64(560.0),
  (8, 5): np.float64(240.0),
  (8, 

### Juega con el embedding

Usando las herramientas proporcionadas en el módulo `minorminor` (en particular la función `minorminer.find_embedding`), encuentra un embedding para el problema que estás resolviendo y resuélvelo sobre la QPU real ajustando los pesos de las cadenas de qubits.